In [1]:
import pickle as pkl
import numpy as np

In [2]:
input_dim = 300

In [3]:
with open('Train_lemmas.txt', 'r', encoding='utf-8') as t:
    train_lem = t.read().split('\n')
t.close()

In [4]:
with open('Val_lemmas.txt', 'r', encoding='utf-8') as t:
    val_lem = t.read().split('\n')
t.close()

In [5]:
with open('Test_lemmas.txt', 'r', encoding='utf-8') as t:
    test_lem = t.read().split('\n')
t.close()

In [6]:
train_lem_count = len(set(' '.join(train_lem).split()))
train_lem_count

121484

In [60]:
sum([len(a.split()) for a in train_lem])/len(train_lem)

38.99672146077137

In [8]:
with open('Train_twenty.pkl', 'rb') as tr:
    X_train_20 = pkl.load(tr)
tr.close()

In [9]:
with open('Val_twenty.pkl', 'rb') as val:
    X_val_20 = pkl.load(val)
val.close()

In [10]:
with open('Test_twenty.pkl', 'rb') as test:
    X_test_20 = pkl.load(test)
test.close()

In [11]:
with open('Train_labels.txt', 'r') as l:
    train_labels = l.read().split('\n')
l.close()
len(train_labels)

87844

In [12]:
with open('Val_labels.txt', 'r') as v:
    val_labels = v.read().split('\n')
v.close()
len(val_labels)

1000

In [13]:
with open('Test_labels.txt', 'r') as t:
    test_labels = t.read().split('\n')
t.close()
len(test_labels)

1000

In [14]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
enc.fit(train_labels)

LabelEncoder()

In [15]:
y_train_labels = enc.transform(train_labels)
y_train_labels.shape

(87844,)

In [16]:
y_val_labels = enc.transform(val_labels)
y_val_labels.shape

(1000,)

In [17]:
y_test_labels = enc.transform(test_labels)
y_test_labels.shape

(1000,)

In [126]:
import keras
from keras import Model, metrics
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Flatten, LSTM, Bidirectional, Input, Concatenate, Conv1D, MaxPooling1D
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [20]:
y_train = to_categorical(y_train_labels)
y_train.shape

(87844, 10)

In [21]:
y_val = to_categorical(y_val_labels)
y_val.shape

(1000, 10)

In [22]:
y_test = to_categorical(y_test_labels)
y_test.shape

(1000, 10)

In [106]:
num_words = 10000
#num_words = train_lem_count

In [107]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_lem)

In [108]:
X_train_300 = tokenizer.texts_to_sequences(train_lem)
X_val_300 = tokenizer.texts_to_sequences(val_lem)
X_test_300 = tokenizer.texts_to_sequences(test_lem)

In [109]:
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
embedding_size = 300

In [110]:
maxlen = 100

X_train_300 = pad_sequences(X_train_300, padding='post', maxlen=maxlen)
X_val_300 = pad_sequences(X_val_300, padding='post', maxlen=maxlen)
X_test_300 = pad_sequences(X_test_300, padding='post', maxlen=maxlen)

In [111]:
from sklearn.utils.class_weight import compute_class_weight

In [112]:
# Instantiate the label encoder
le = LabelEncoder()

# Fit the label encoder to our label series
le.fit(list(y_train_labels))

# Create integer based labels Series
y_integers = le.transform(list(y_train_labels))

In [113]:
class_weights = compute_class_weight('balanced', np.unique(y_train_labels), y_train_labels)
class_weights_dict = dict(zip(le.transform(list(le.classes_)), class_weights))

In [114]:
# Parameters
batch_size = 512
epochs = 100
num_classes = 10

In [128]:
inputA = Input(shape=(maxlen,))
inputB = Input(shape=(20,))

In [129]:
emb = Embedding(output_dim=embedding_size, input_dim=num_words, input_length=maxlen, trainable = True)(inputA)
x = Dropout(0.2)(emb)
x = Conv1D(26, 10, activation='relu')(x)
x = MaxPooling1D(2)(x)
x = LSTM(200)(emb)
x = Model(inputs = inputA, outputs = x)

In [130]:
y = Dense(20, activation="relu")(inputB)
y = Dense(10, activation="relu")(y)
y = Model(inputs=inputB, outputs = y)

In [131]:
combined = Concatenate()([x.output, y.output])

In [132]:
z = Dense(200, activation='relu')(combined)
z = Dense(100, activation='relu')(combined)
z = Dense(num_classes, activation='softmax')(z)

In [133]:
model = Model(inputs =[x.input, y.input], outputs = z)

In [134]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=[metrics.categorical_accuracy])
print(model.summary())

Model: "model_21"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 100, 300)     3000000     input_15[0][0]                   
__________________________________________________________________________________________________
dense_31 (Dense)                (None, 20)           420         input_16[0][0]                   
___________________________________________________________________________________________

In [135]:
earlystop = EarlyStopping(monitor='val_categorical_accuracy', min_delta = 0.0001, patience=5, verbose=1, mode='auto')
callbacks_list = [earlystop]

In [136]:
model.fit([X_train_300, X_train_20], y_train, batch_size=batch_size, epochs=epochs, callbacks=callbacks_list,
          validation_data=([X_val_300, X_val_20], y_val), class_weight = class_weights_dict, verbose=1)

Train on 87844 samples, validate on 1000 samples
Epoch 1/100
87844/87844 [==============================] - 466s 5ms/step - loss: 2.1287 - categorical_accuracy: 0.3234 - val_loss: 1.6736 - val_categorical_accuracy: 0.4280
Epoch 2/100
87844/87844 [==============================] - 354s 4ms/step - loss: 1.5776 - categorical_accuracy: 0.5511 - val_loss: 1.3915 - val_categorical_accuracy: 0.5510
Epoch 3/100
87844/87844 [==============================] - 347s 4ms/step - loss: 1.2957 - categorical_accuracy: 0.6202 - val_loss: 1.0132 - val_categorical_accuracy: 0.7030
Epoch 4/100
87844/87844 [==============================] - 344s 4ms/step - loss: 0.8252 - categorical_accuracy: 0.7933 - val_loss: 0.6966 - val_categorical_accuracy: 0.7850
Epoch 5/100
87844/87844 [==============================] - 339s 4ms/step - loss: 0.5337 - categorical_accuracy: 0.8614 - val_loss: 0.6283 - val_categorical_accuracy: 0.8130
Epoch 6/100
87844/87844 [==============================] - 337s 4ms/step - loss: 0.372

In [137]:
scores = model.evaluate([X_val_300, X_val_20], y_val, verbose=0)
print('Val loss:', scores[0])
print('Val accuracy:', scores[1])

Val loss: 1.0082062370032072
Val accuracy: 0.8


In [138]:
y_pred = model.predict([X_val_300, X_val_20])
y_pred_classes = [[1 if c == max(a) else 0 for c in a] for a in y_pred]
y_pred_labels = [a.index(1) for a in y_pred_classes]
confusion_matrix(y_pred_labels, y_val_labels)
#y_val_labels

array([[71,  0,  1,  5,  0,  0,  4,  5,  1,  1],
       [ 3, 67,  3,  0,  0,  0,  0,  1,  3,  2],
       [ 3,  7, 88,  3,  0,  0,  0,  1,  3,  1],
       [ 9,  5,  3, 84,  2,  0,  3,  2,  2,  1],
       [ 1,  3,  1,  2, 87,  0,  0,  1,  7,  2],
       [ 3,  0,  0,  1,  1, 89,  1,  0,  2,  2],
       [ 6,  1,  1,  1,  2,  9, 88,  0,  2,  0],
       [ 3, 11,  0,  3,  5,  1,  3, 90,  9,  5],
       [ 0,  2,  0,  0,  1,  0,  0,  0, 56,  6],
       [ 1,  4,  3,  1,  2,  1,  1,  0, 15, 80]])

In [139]:
test_scores = model.evaluate([X_test_300, X_test_20], y_test, verbose=0)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

Test loss: 1.0909465174674988
Test accuracy: 0.79


In [140]:
y_pred = model.predict([X_test_300, X_test_20])
y_pred_classes = [[1 if c == max(a) else 0 for c in a] for a in y_pred]
y_pred_labels = [a.index(1) for a in y_pred_classes]
confusion_matrix(y_pred_labels, y_test_labels)

array([[81,  7,  2, 12,  0,  0,  0,  2,  2,  1],
       [ 1, 68,  3,  1,  0,  0,  0,  0,  5,  3],
       [ 1,  6, 80,  2,  0,  0,  0,  0,  1,  2],
       [ 3,  5,  8, 72,  1,  0,  1,  4,  0,  4],
       [ 3,  4,  2,  0, 89,  0,  0,  4, 13,  2],
       [ 0,  2,  0,  3,  1, 98,  3,  0,  0,  3],
       [ 8,  1,  3,  4,  1,  1, 89,  3,  1,  0],
       [ 2,  2,  2,  5,  1,  0,  6, 85,  8,  1],
       [ 0,  4,  0,  0,  5,  0,  0,  1, 50,  6],
       [ 1,  1,  0,  1,  2,  1,  1,  1, 20, 78]])

In [141]:
print(classification_report(y_pred_labels, y_test_labels))

              precision    recall  f1-score   support

           0       0.81      0.76      0.78       107
           1       0.68      0.84      0.75        81
           2       0.80      0.87      0.83        92
           3       0.72      0.73      0.73        98
           4       0.89      0.76      0.82       117
           5       0.98      0.89      0.93       110
           6       0.89      0.80      0.84       111
           7       0.85      0.76      0.80       112
           8       0.50      0.76      0.60        66
           9       0.78      0.74      0.76       106

    accuracy                           0.79      1000
   macro avg       0.79      0.79      0.79      1000
weighted avg       0.81      0.79      0.79      1000



In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot cnn.png', show_shapes=True, show_layer_names=True)